In [1]:
import numpy as np
import pandas as pd

In [2]:
bus_lab = pd.read_csv('bus_lab.csv')
lab_bus = pd.read_csv('lab_bus.csv')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#print shape of lab_bus and bus_lab
print(lab_bus.shape)
print(bus_lab.shape)

#Replace mnc with MNC in Actor1Code and Actor2Code
bus_lab['Actor1Code'] = bus_lab['Actor1Code'].replace('mnc', 'MNC') #replace mnc with MNC in Actor1Code
lab_bus['Actor2Code'] = lab_bus['Actor2Code'].replace('mnc', 'MNC') #replace mnc with MNC in Actor2Code

#Drop duplicates in bus_lab and lab_bus based on GLOBALEVENTID
bus_lab = bus_lab.drop_duplicates(subset=['GLOBALEVENTID','GoldsteinScale','Year']) #drop duplicates in bus_lab based on GLOBALEVENTID
lab_bus = lab_bus.drop_duplicates(subset=['GLOBALEVENTID','GoldsteinScale','Year']) #drop duplicates in lab_bus based on GLOBALEVENTID

#Rename column 'GoldsteinScale' to 'Business-Labor Goldstein Score' in bus_lab
bus_lab = bus_lab.rename(columns={'GoldsteinScale': 'Business->Labor Goldstein Score'})
#Rename column 'GoldsteinScale' to 'Labor-Business Goldstein Score' in lab_bus
lab_bus = lab_bus.rename(columns={'GoldsteinScale': 'Labor->Business Goldstein Score'})

#Reset index of bus_lab and lab_bus
bus_lab = bus_lab.reset_index(drop=True) #reset index of bus_lab
lab_bus = lab_bus.reset_index(drop=True) #reset index of lab_bus

print(lab_bus['Actor1Code'].unique())
print(bus_lab['Actor1Code'].unique())
print(lab_bus['Actor2Code'].unique())
print(bus_lab['Actor2Code'].unique())

#print shape of lab_bus and bus_lab
print(lab_bus.shape)
print(bus_lab.shape)

(754080, 59)
(1002278, 59)
['LAB']
['BUS' 'MNC']
['BUS' 'MNC']
['LAB']
(294758, 59)
(386544, 59)


In [4]:
def lat_long_to_PG_lat_long(x, list_lat_long):
    for i in range(len(x)):
        if x[i] >= 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) + 0.25)
            else:
                list_lat_long.append(int(x[i]) + 0.75)

        if x[i] < 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) - 0.75)
            else:
                list_lat_long.append(int(x[i]) - 0.25)


pg_lat = []
pg_long = []
lat_long_to_PG_lat_long(lab_bus["ActionGeo_Lat"], pg_lat)
lat_long_to_PG_lat_long(lab_bus["ActionGeo_Long"], pg_long)
lab_bus['pg_lat'] = pg_lat
lab_bus['pg_long'] = pg_long

pg_lat = []
pg_long = []
lat_long_to_PG_lat_long(bus_lab["ActionGeo_Lat"], pg_lat)
lat_long_to_PG_lat_long(bus_lab["ActionGeo_Long"], pg_long)
bus_lab['pg_lat'] = pg_lat
bus_lab['pg_long'] = pg_long

In [5]:
prio_grid = pd.read_csv('../PRIO GRID spine.csv') #read in PRIO GRID spine file
year_df = pd.DataFrame({'year':range(1990,2022)}) #create a dataframe with years 1990 to 2021
year_df['key'] = 1
prio_grid['key'] = 1
base_df = pd.merge(year_df, prio_grid, on ='key').drop("key", 1)
base_df

,year,gid,lon,lat
0,1990,49182,-69.25,-55.75
1,1990,49183,-68.75,-55.75
2,1990,49184,-68.25,-55.75
3,1990,49185,-67.75,-55.75
4,1990,49186,-67.25,-55.75
...,...,...,...,...
2074171,2021,249340,-70.25,83.25
2074172,2021,249341,-69.75,83.25
2074173,2021,249342,-69.25,83.25
2074174,2021,249343,-68.75,83.25


In [6]:
#Merge GoldsteinScale column from lab_bus with base_df on pg_lat and pg_long into one new dataframe df
df = pd.merge(base_df, lab_bus[['pg_lat','pg_long','Labor->Business Goldstein Score','GLOBALEVENTID','Year']], how = 'left', left_on = ['lat','lon','year'], right_on = ['pg_lat','pg_long','Year']).drop(['pg_lat','pg_long','Year'], 1) #merge lab_bus with base_df
df = df.sort_values(by=['gid','year']).reset_index(drop=True) #sort df by gid

In [7]:
#Group by year and gid and calculate the mean of GoldsteinScale
lab_bus_final = df.groupby(['year','gid']).mean().reset_index() #group by year and gid and calculate the mean of GoldsteinScale
lab_bus_final

,year,gid,lon,lat,Labor->Business Goldstein Score,GLOBALEVENTID
0,1990,49182,-69.25,-55.75,NaN,NaN
1,1990,49183,-68.75,-55.75,NaN,NaN
2,1990,49184,-68.25,-55.75,NaN,NaN
3,1990,49185,-67.75,-55.75,NaN,NaN
4,1990,49186,-67.25,-55.75,NaN,NaN
...,...,...,...,...,...,...
2074171,2021,249340,-70.25,83.25,NaN,NaN
2074172,2021,249341,-69.75,83.25,NaN,NaN
2074173,2021,249342,-69.25,83.25,NaN,NaN
2074174,2021,249343,-68.75,83.25,NaN,NaN


In [8]:
#Merge GoldsteinScale column from bus_lab with base_df on pg_lat and pg_long into one new dataframe df
df = pd.merge(base_df, bus_lab[['pg_lat','pg_long','Business->Labor Goldstein Score','GLOBALEVENTID','Year']], how = 'left', left_on = ['lat','lon','year'], right_on = ['pg_lat','pg_long','Year']).drop(['pg_lat','pg_long','Year'], 1) #merge bus_lab with base_df
df = df.sort_values(by=['gid','year']).reset_index(drop=True) #sort df by gid

In [9]:
#Group by year and gid and calculate the mean of GoldsteinScale
bus_lab_final = df.groupby(['year','gid']).mean().reset_index() #group by year and gid and calculate the mean of GoldsteinScale
bus_lab_final

,year,gid,lon,lat,Business->Labor Goldstein Score,GLOBALEVENTID
0,1990,49182,-69.25,-55.75,NaN,NaN
1,1990,49183,-68.75,-55.75,NaN,NaN
2,1990,49184,-68.25,-55.75,NaN,NaN
3,1990,49185,-67.75,-55.75,NaN,NaN
4,1990,49186,-67.25,-55.75,NaN,NaN
...,...,...,...,...,...,...
2074171,2021,249340,-70.25,83.25,NaN,NaN
2074172,2021,249341,-69.75,83.25,NaN,NaN
2074173,2021,249342,-69.25,83.25,NaN,NaN
2074174,2021,249343,-68.75,83.25,NaN,NaN


In [10]:
#Merge bus_lab_final and lab_bus_final into one dataframe
bus_lab_lab_bus = pd.merge(bus_lab_final, lab_bus_final, how = 'inner', on = ['year','gid']) #merge bus_lab_final and lab_bus_final

In [11]:
bus_lab_lab_bus = bus_lab_lab_bus.rename(columns={'gid': 'PRIO GRID', 'year': 'Year'}) #rename gid to PRIO GRID, year to Year
bus_lab_lab_bus = bus_lab_lab_bus[['PRIO GRID','Year','Business->Labor Goldstein Score','Labor->Business Goldstein Score']] #reorder columns
bus_lab_lab_bus = bus_lab_lab_bus.sort_values(by=['PRIO GRID','Year']).reset_index(drop=True) #sort by PRIO GRID and Year

In [12]:
bus_lab_lab_bus.shape

(2074176, 4)

In [14]:
bus_lab_lab_bus.to_csv('Goldstein Score for Business and Labor.csv', index=False) #save bus_lab_lab_bus to csv